# Plane Flying Game
by Sahil Dev x GitHub Copilot

## Objective
Your goal is to fly the plane without running into obstacles.

### Imports, Constants, and Utilities

In [1]:
# imports
import numpy as np
import pygame
import pygame_menu
from itertools import repeat
import os
import sys

if getattr(sys, 'frozen', False):
    os.chdir(sys._MEIPASS)

# constants
SCREEN_WIDTH = 1500
SCREEN_HEIGHT = 1080
SCREEN_SIZE = (SCREEN_WIDTH, SCREEN_HEIGHT)
GROUND_COLOR = (225, 185, 175)
SKY_COLOR = (100, 225, 255)
SPACE_COLOR = (6, 0, 18)
DEEP_SPACE_COLOR = (80, 0, 60)
NEBULA_COLOR = (0, 55, 50)
SKY_LEVEL = 9
SPACE_LEVEL = 16
DEEP_SPACE_LEVEL = 22
# NEUBULA_LEVEL = 22

BASIC_TITLE = 'Private'
SKY_TITLE = 'Lieutenant'
SPACE_TITLE = 'Space Commander'
DEEP_SPACE_TITLE = 'Galactic General'
BASIC_TITLE_ABV = 'Pvt.'
SKY_TITLE_ABV = 'Lt.'
SPACE_TITLE_ABV = 'Space Cdr.'
DEEP_SPACE_TITLE_ABV = 'Galactic Gen.'

DEATH_ANIMATION_TIME = 250 # for enemies
FPS = 60

# global variables
original_screen = None
screen = None
clock = None
menu = None
player_name = 'Player Name'
screen_offset = repeat((0, 0))

# utilities
def setplayername(name):
    global player_name
    player_name = name

# return the title of the player
def get_title(level, abbreviate=False):
    if abbreviate:
        if level < SKY_LEVEL:
            return BASIC_TITLE_ABV
        elif level < SPACE_LEVEL:
            return SKY_TITLE_ABV
        elif level < DEEP_SPACE_LEVEL:
            return SPACE_TITLE_ABV
        else:
            return DEEP_SPACE_TITLE_ABV

    if level < SKY_LEVEL:
        return BASIC_TITLE
    elif level < SPACE_LEVEL:
        return SKY_TITLE
    elif level < DEEP_SPACE_LEVEL:
        return SPACE_TITLE
    else:
        return DEEP_SPACE_TITLE

# add player title to name
def add_title(name, level, abbreviate=False):
    title = get_title(level, abbreviate=abbreviate)
    return f'{title} {name}'

# shake the screen (on hit)
def shake(duration=50, amplitude=10, damping=1, frequency=1):
    for i in range(0, duration):
        t = i / 15
        d = -damping * t
        f = 2 * np.pi * t * frequency
        x = amplitude * np.exp(d) * np.cos(f)
        yield (x, 0)
    while True:
        yield (0, 0)

def lerp_color(color1, color2, t):
    # clamp t between 0 and 1
    t = max(0, min(1, t))
    return tuple(int(color1[i] + (color2[i] - color1[i]) * t) for i in range(3))

# get the background color for a level
def get_background_color(level):
    if level < SKY_LEVEL:
        return lerp_color(GROUND_COLOR, SKY_COLOR, (level - 1) / (SKY_LEVEL - 1))
    elif level < SPACE_LEVEL:
        return lerp_color(SKY_COLOR, SPACE_COLOR, (level - SKY_LEVEL) / (SPACE_LEVEL - SKY_LEVEL))
    elif level <= DEEP_SPACE_LEVEL:
        return lerp_color(SPACE_COLOR, DEEP_SPACE_COLOR, (level - SPACE_LEVEL) / (DEEP_SPACE_LEVEL - SPACE_LEVEL))
    else:
        return DEEP_SPACE_COLOR

# get the background color
def get_background_colors(level):
    if level >= DEEP_SPACE_LEVEL:
        return get_background_color(DEEP_SPACE_LEVEL - 1), NEBULA_COLOR
    else:
        return get_background_color(level - 1), get_background_color(level + 1)

# get stars for a given level
def get_stars(level):
    # decide how many stars to draw
    if level >= SPACE_LEVEL - 1:
        num_stars = 80 + 60 * (level - SPACE_LEVEL)
    else:
        num_stars = 0
    
    stars = []
    for i in range(num_stars):
        # generate a random star with color, position, and radius
        color = (np.random.randint(230, 255), np.random.randint(230, 240), np.random.randint(220, 245))
        
        # random position more likely to be on the top of the screen
        r = np.random.rand()
        if r < 0.5:
            y = np.random.randint(0, SCREEN_HEIGHT // 3)
        elif r < 0.7:
            y = np.random.randint(SCREEN_HEIGHT // 3, SCREEN_HEIGHT // 2)
        else:
            y = np.random.randint(SCREEN_HEIGHT // 2, SCREEN_HEIGHT)

        pos = (np.random.randint(0, SCREEN_WIDTH), y)
        radius = np.random.randint(1, 3)
        stars.append((color, pos, radius))
    
    return stars

def vertical_gradient(size, startcolor, endcolor):
    """
    Draws a vertical linear gradient filling the entire surface. Returns a
    surface filled with the gradient (numeric is only 2-3 times faster).
    """
    height = size[1]
    bigSurf = pygame.Surface((1,height)).convert_alpha()
    dd = 1.0/height

    if len(startcolor) == 3:
        sr, sg, sb = startcolor
        er, eg, eb = endcolor
        sa, ea = 255, 255
    elif len(startcolor) == 4:
        sr, sg, sb, sa = startcolor
        er, eg, eb, ea = endcolor

    rm = (er-sr)*dd
    gm = (eg-sg)*dd
    bm = (eb-sb)*dd
    am = (ea-sa)*dd
    
    for y in range(height):
        bigSurf.set_at((0,y),
                        (int(sr + rm*y),
                         int(sg + gm*y),
                         int(sb + bm*y),
                         int(sa + am*y))
                      )
    return pygame.transform.scale(bigSurf, size)

def get_gradient_and_font(level):
    bottom_color, top_color = get_background_colors(level)
    gradient = vertical_gradient(screen.get_size(), top_color, bottom_color)

    if sum(top_color) > 300:
        fontcolor = (15, 15, 15)
    else:
        fontcolor = (240, 240, 240)

    return gradient, fontcolor

# load top players and scores from scores.txt
def load_scores():
    scores = []
    with open('scores.txt', 'r') as f:
        for line in f:
            result = line.split()
            score = int(result[-1])
            name = ' '.join(result[:-1])
            scores.append((name, score))
    return scores

# insort a score into scores list
def insort_score(name, score, scores):
    if len(scores) == 0:
        scores.append([name, score])
        return 0

    scores.sort(key=lambda x: x[1], reverse=True)
    for i in range(len(scores)):
        if score > int(scores[i][1]):
            scores.insert(i, [name, score])
            return i
    
    scores.append([name, score])
    return len(scores) - 1

# save top scores to scores.txt
def save_scores(scores):
    scores = scores[:10]
    with open('scores.txt', 'w') as f:
        for score in scores:
            f.write(f'{score[0]} {score[1]}\n')
    return scores

# return true if arrow keys, space, enter, or letter keys are pressed
def check_for_keypress():
    got_key = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            quit()

    press=pygame.key.get_pressed()
    if press[pygame.K_LEFT] or press[pygame.K_RIGHT] or press[pygame.K_UP] or press[pygame.K_DOWN]:
        got_key = True
    
    if press[pygame.K_SPACE] or press[pygame.K_RETURN]:
        got_key = True

    for key in range(pygame.K_a, pygame.K_z + 1):
        if press[key]:
            got_key = True

    return got_key

# wait for keypress after one second
def wait_for_keypress():
    time = pygame.time.get_ticks()
    delay = 600

    # wait for keypress and delay
    while not check_for_keypress() or pygame.time.get_ticks() - time < delay:
        pass

pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame-menu 4.2.2


### Main Game Loop

In [2]:
def start_game():
    # initialize level
    level = 1
    # initialize player
    global player_name, original_screen, screen_offset
    player = Player(player_name)
    paused = False

    # initialize dying enemies
    dying_enemies = []

    # initialize game loop
    while player.lives > 0:
        # load player sprite
        player.load_image(level)

        # initialize enemies
        num_enemies = level * 2 - 1
        enemies = []
        for i in range(num_enemies):
            enemies.append(Enemy(level))

        # initialize bullets
        bullets = []

        # get a list of the stars to draw
        stars = get_stars(level)
        
        while len(enemies) > 0 and player.lives > 0:
            # handle events
            events = pygame.event.get()
            for event in events:
                if event.type == pygame.QUIT:
                    return
                elif event.type == pygame_menu.events.BACK:
                    return
                elif event.type == pygame.KEYDOWN:
                    # pause game if escape is pressed
                    if event.key == pygame.K_ESCAPE:
                        paused = True
            
            # resume if any key is pressed
            if paused and check_for_keypress():
                paused = False

            if not paused:
                # update game
                player.update(events, bullets)
                for enemy in enemies:
                    enemy.update(player, level)

                # iterate over bullets and delete if necessary
                i = 0
                while i < len(bullets):
                    bullet = bullets[i]
                    should_destroy = bullet.update()
                    if should_destroy:
                        bullets.pop(i)
                    else:
                        i += 1

                # check for collisions
                for enemy_index, enemy in enumerate(enemies):
                    for bullet_index, bullet in enumerate(bullets):
                        if bullet.rect.colliderect(enemy.rect):
                            bullets.pop(bullet_index)
                            
                            # add a new dying enemy, with the time of death
                            dying_enemies.append((enemies[enemy_index], pygame.time.get_ticks()))

                            enemies.pop(enemy_index)
                            player.score += 1
                            break
                    
                    # check if player collided with enemy
                    if enemy.rect.colliderect(player.rect):
                        player.lives -= 1

                        # shake the screen
                        screen_offset = shake(duration=50, amplitude=10, damping=1, frequency=1)

                        enemy.revive(level)
                        if player.lives <= 0:
                            break

            # get the background colors
            bottom_color, top_color = get_background_colors(level)

            # draw game
            gradient = vertical_gradient(screen.get_size(), top_color, bottom_color)
            screen.blit(gradient, (0, 0))

            # draw stars
            for star in stars:
                color = star[0]
                pos = star[1]
                radius = star[2]

                # draw star
                pygame.draw.circle(screen, color, pos, radius)

            # draw player
            player.draw(screen)

            # draw enemies
            for enemy in enemies:
                enemy.draw(screen)
                
            # draw dying enemies
            for i, (enemy, time_of_death) in enumerate(dying_enemies):
                if pygame.time.get_ticks() - time_of_death > DEATH_ANIMATION_TIME:
                    dying_enemies.pop(i)
                else:
                    enemy.draw(screen, alpha=255 * (1 - (pygame.time.get_ticks() - time_of_death) / DEATH_ANIMATION_TIME))

            # draw bullets
            for bullet in bullets:
                bullet.draw(screen)

            # set fontcolor to light/dark depending on background brightness
            if sum(top_color) > 300:
                fontcolor = (15, 15, 15)
            else:
                fontcolor = (240, 240, 240)

            # draw loading bar for special attack
            completion = (pygame.time.get_ticks() - player.last_special_attack) / player.special_attack_delay

            # clamp completion to [0, 1]
            completion = max(0, min(1, completion))

            # draw loading bar below player
            bar_width = player.rect.width * 0.8
            # loading bar background
            pygame.draw.rect(screen, (0, 0, 0), (player.rect.centerx - bar_width / 2, player.rect.bottom + 10, bar_width, 8))
            # loading bar foreground
            # make cyan if bar is full
            if completion == 1: 
                color = (120, 240, 255)
            else:
                # interpolate between green and red
                # g = (100, 240, 120)
                # r = (240, 120, 100)
                # color = lerp_color(r, g, completion)
                color = (0, 200, 200)
                
            completion_width = int(bar_width * completion)
            pygame.draw.rect(screen, color, (player.rect.centerx - bar_width / 2, player.rect.bottom + 10, completion_width, 8))
            
            # display player score in top left corner
            score_text = pygame.font.Font(None, 50).render(f'Score: {player.score}', True, fontcolor)
            score_rect = score_text.get_rect()
            score_rect.topleft = (10, 10)
            screen.blit(score_text, score_rect)

            # display level in top center
            level_text = pygame.font.Font(None, 60).render(f'Level {level}', True, fontcolor)
            level_rect = level_text.get_rect()
            level_rect.center = (SCREEN_WIDTH // 2, 40)
            screen.blit(level_text, level_rect)

            # display title and player name in bottom center
            name_text = pygame.font.Font(None, 50).render(add_title(player.name, level), True, fontcolor)
            name_rect = name_text.get_rect()
            name_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT - 40)
            screen.blit(name_text, name_rect)

            # display player lives in top right corner
            lives_text = pygame.font.Font(None, 50).render(f'Lives: {player.lives}', True, fontcolor)
            lives_rect = lives_text.get_rect()
            lives_rect.topright = (SCREEN_WIDTH - 10, 10)
            screen.blit(lives_text, lives_rect)

            # display pause message if paused
            if paused:
                paused_text = pygame.font.Font(None, 80).render('Game Paused', True, fontcolor)
                paused_rect = paused_text.get_rect()
                paused_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
                screen.blit(paused_text, paused_rect)

                # press any key to resume
                resume_text = pygame.font.Font(None, 50).render('(press any key to resume)', True, fontcolor)
                resume_rect = resume_text.get_rect()
                resume_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 100)
                screen.blit(resume_text, resume_rect)

            original_screen.blit(screen, next(screen_offset))
            pygame.display.update()
            clock.tick(FPS)
        
        if player.lives > 0:
            # level up
            level += 1
            player.lives += 1

    lose_screen(player, level)
    
def lose_screen(player, level):
    # lose screen
    gradient, fontcolor = get_gradient_and_font(level)
    original_screen.blit(gradient, (0, 0))

    stars = get_stars(level)

    # draw stars
    for star in stars:
        color = star[0]
        pos = star[1]
        radius = star[2]

        # draw star
        pygame.draw.circle(original_screen, color, pos, radius)

    # lose text
    lose_text = pygame.font.Font(None, 80).render('Game Over', True, fontcolor)
    lose_rect = lose_text.get_rect()
    lose_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 100)
    original_screen.blit(lose_text, lose_rect)
    
    # score text
    score_text = pygame.font.Font(None, 50).render(f'Score: {player.score}', True, fontcolor)
    score_rect = score_text.get_rect()
    score_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 20)
    original_screen.blit(score_text, score_rect)

    # press any key to continue
    press_any_key_text = pygame.font.Font(None, 30).render('(press any key to continue)', True, fontcolor)
    press_any_key_rect = press_any_key_text.get_rect()
    press_any_key_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT - 40)
    original_screen.blit(press_any_key_text, press_any_key_rect)

    pygame.display.update()
    
    # add to highscore list
    print('Loading scores...')
    scores = load_scores()
    score = player.score
    titled_name = add_title(player.name, level, abbreviate=True)
    place = insort_score(titled_name, score, scores)

    # save high scores
    print(f'Saving {len(scores)} scores...')
    save_scores(scores)
    
    # wait for keypress
    wait_for_keypress()

    # move to high scores screen
    view_high_scores(scores=scores, place=place, level=level)

def view_high_scores(scores=None, place=None, level=SPACE_LEVEL):
    if scores is None:
        scores = load_scores()
    
    # high score screen
    gradient, fontcolor = get_gradient_and_font(level)
    original_screen.blit(gradient, (0, 0))

    stars = get_stars(level)

    # draw stars
    for star in stars:
        color = star[0]
        pos = star[1]
        radius = star[2]
        
        # draw star
        pygame.draw.circle(original_screen, color, pos, radius)

    # display high scores
    high_scores_text = pygame.font.Font(None, 100).render('High Scores', True, fontcolor)
    high_scores_rect = high_scores_text.get_rect()
    high_scores_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 330)
    original_screen.blit(high_scores_text, high_scores_rect)

    for i, score in enumerate(scores):
        name, score = score

        # if current player, display in red
        if i == place:
            r = min(fontcolor[0] + 100, 255)
            g = max(fontcolor[1] - 80, 0)
            b = max(fontcolor[2] - 80, 0)
            color = (r, g, b)
        else:
            color = fontcolor

        # display place, name, and score in columns
        place_text = pygame.font.Font(None, 60).render(f'{i + 1}.', True, color)
        place_rect = place_text.get_rect()
        place_rect.midright = (SCREEN_WIDTH // 2 - 400, SCREEN_HEIGHT // 2 - 240 + i * 55)
        original_screen.blit(place_text, place_rect)

        name_text = pygame.font.Font(None, 60).render(name, True, color)
        name_rect = name_text.get_rect()
        name_rect.midleft = (SCREEN_WIDTH // 2 - 380, SCREEN_HEIGHT // 2 - 240 + i * 55)
        original_screen.blit(name_text, name_rect)

        score_text = pygame.font.Font(None, 60).render(f'{score}', True, color)
        score_rect = score_text.get_rect()
        score_rect.midleft = (SCREEN_WIDTH // 2 + 400, SCREEN_HEIGHT // 2 - 240 + i * 55)
        original_screen.blit(score_text, score_rect)

    # press any key to continue
    press_any_key_text = pygame.font.Font(None, 30).render('(press any key to continue)', True, fontcolor)
    press_any_key_rect = press_any_key_text.get_rect()
    press_any_key_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT - 40)
    original_screen.blit(press_any_key_text, press_any_key_rect)
    
    pygame.display.update()
    
    # wait for keypress
    wait_for_keypress()

    # return to main menu
    return

# display controls
def view_controls(level=SPACE_LEVEL):
    # controls screen
    gradient, fontcolor = get_gradient_and_font(level)
    original_screen.blit(gradient, (0, 0))

    stars = get_stars(level)

    # draw stars
    for star in stars:
        color = star[0]
        pos = star[1]
        radius = star[2]

        # draw star
        pygame.draw.circle(original_screen, color, pos, radius)

    # display controls
    controls_text = pygame.font.Font(None, 100).render('Controls', True, fontcolor)
    controls_rect = controls_text.get_rect()
    controls_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 180)
    original_screen.blit(controls_text, controls_rect)

    # display controls
    controls_text = pygame.font.Font(None, 50).render('Use the left and right arrow keys to move', True, fontcolor)
    controls_rect = controls_text.get_rect()
    controls_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 60)
    original_screen.blit(controls_text, controls_rect)

    controls_text = pygame.font.Font(None, 50).render('SPACE to shoot', True, fontcolor)
    controls_rect = controls_text.get_rect()
    controls_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)
    original_screen.blit(controls_text, controls_rect)

    controls_text = pygame.font.Font(None, 50).render('ESCAPE to pause', True, fontcolor)
    controls_rect = controls_text.get_rect()
    controls_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 60)
    original_screen.blit(controls_text, controls_rect)

    press_any_key_text = pygame.font.Font(None, 30).render('(press any key to continue)', True, fontcolor)
    press_any_key_rect = press_any_key_text.get_rect()
    press_any_key_rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT - 40)
    original_screen.blit(press_any_key_text, press_any_key_rect)

    pygame.display.update()

    # wait for keypress
    wait_for_keypress()

    # return to main menu
    return

### Player, Enemy, and Bullet Classes

In [3]:
# Player class
class Player:
    def __init__(self, name):
        self.load_image(1)

        self.name = name
        self.acceleration = 2
        self.deceleration = 0.6
        self.velocity = 0
        self.max_speed = 18

        # movement and shooting variables
        self.direction = 0
        self.shoot_delay = 0
        self.shoot_delay_held = 200
        self.last_shot = pygame.time.get_ticks()
        self.special_attack_delay = 8000
        self.last_special_attack = pygame.time.get_ticks()

        self.score = 0
        self.lives = 5

    def load_image(self, level):
        # set size based on level, 180x180 starting at level 1
        size = max(180 - (level - SKY_LEVEL) * 6, 52)
            
        self.image = pygame.image.load('images/airplane.png')
        self.image = pygame.transform.scale(self.image, (size, size))
        self.image = self.image.convert_alpha()

        if level > 1:
            midbottom = self.rect.midbottom

        self.rect = self.image.get_rect()

        if level > 1:
            self.rect.midbottom = midbottom
        else:
            self.rect.midbottom = SCREEN_WIDTH // 2, SCREEN_HEIGHT - 100
        

    def shoot(self, bullets):
        global screen_offset

        now = pygame.time.get_ticks()

        # special shot
        if now - self.last_special_attack > self.special_attack_delay:
            self.last_special_attack = now
            self.last_shot = now

            # shoot seven shots in an arc
            for i in range(7):
                y_dir = -1
                # interpolate x from -1 to 1
                x_dir = -1 + 2 * i / 6
                b = Bullet(self.rect.center, direction = (x_dir, y_dir), size=140)
                bullets.append(b)
            
            # screen shake
            screen_offset = shake(duration=100, amplitude=50, damping=0.7, frequency=3)

        # normal shot
        elif now - self.last_shot > self.shoot_delay:
            self.last_shot = now
            b = Bullet(self.rect.center)
            bullets.append(b)
    
    def shoot_held(self, bullets):
        now = pygame.time.get_ticks()
        if now - self.last_shot > self.shoot_delay_held:
            self.last_shot = now
            b = Bullet(self.rect.center)
            bullets.append(b)

    def update(self, events, bullets):
        global screen
        
        self.direction = 0
        shoot = False

        for event in events:
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    shoot = True

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.direction -= 1
        if keys[pygame.K_RIGHT]:
            self.direction += 1
        if keys[pygame.K_SPACE] and not shoot:
            self.shoot_held(bullets)

        if shoot:
            self.shoot(bullets)

        # accelerate player
        if self.direction == -1:
            self.velocity -= self.acceleration
        elif self.direction == 1:
            self.velocity += self.acceleration
        # slow player down if no key is pressed
        elif self.direction == 0:
            self.velocity *= self.deceleration
        
        # limit player speed to max speed
        if self.velocity > self.max_speed:
            self.velocity = self.max_speed
        elif self.velocity < -self.max_speed:
            self.velocity = -self.max_speed
        
        # round to nearest integer
        update = round(self.velocity)
        self.rect.x += update
        
        # clamp player to screen
        self.rect.clamp_ip(screen.get_rect())
        
    def draw(self, screen):
        screen.blit(self.image, self.rect)

# Bullet class
class Bullet:
    def __init__(self, pos, direction=None, size=None):

        if size is None:
            size = 60

        # load image in at 60x60
        self.image = pygame.image.load('images/bullet.png')
        self.image = pygame.transform.scale(self.image, (size, size))
        self.image = self.image.convert_alpha()

        self.rect = self.image.get_rect()
        self.rect.center = pos
        self.speed = 10
        if direction is not None:
            self.direction = direction
        else:
            self.direction = (0, -1)
        
    def update(self):
        """
        Return True if the bullet is ready to be destroyed.
        """
        self.rect.x += int(self.direction[0] * self.speed)
        self.rect.y += int(self.direction[1] * self.speed)
        if self.rect.top < 0:
            return True
        return False

    def draw(self, screen):
        screen.blit(self.image, self.rect)

# Enemy class
class Enemy:
    def __init__(self, level):
        # set size based on level, 85x85 starting at level 1
        size = max(85 - (level - 1) * 3, 20)
            
        self.rect = pygame.Rect(0, 0, size, size)
        self.rect.centerx = np.random.randint(0, SCREEN_WIDTH)
        self.rect.centery = np.random.randint(0, SCREEN_HEIGHT // 4)

        # clamp enemy to screen
        self.rect.clamp_ip(screen.get_rect())
        
        # set speed based on level
        if level >= SKY_LEVEL:
            self.speed = np.random.randint(3, 3 + level // 2)
        else:
            self.speed = np.random.randint(3, 6)
        self.color = (np.random.randint(0, 220), np.random.randint(0, 220), np.random.randint(0, 220))

    def update(self, player, level):
        self.rect.centery += self.speed
        if self.rect.top > SCREEN_HEIGHT:
            self.revive(level)

    # revive with higher speed
    def revive(self, level):
        self.rect.centerx = np.random.randint(0, SCREEN_WIDTH)
        self.rect.centery = np.random.randint(0, SCREEN_HEIGHT // 4)

        # clamp enemy to screen
        self.rect.clamp_ip(screen.get_rect())

        # increase speed of enemies up to level + 4
        self.speed += 2
        self.speed = min(self.speed, level + 4)

        self.color = (np.random.randint(0, 220), np.random.randint(0, 220), np.random.randint(0, 220))

    def draw(self, screen, alpha=255):
        # create a new surface with the same size as the enemy
        surface = pygame.Surface(self.rect.size)
        surface.fill(self.color)
        surface.set_alpha(alpha)
        screen.blit(surface, self.rect)

### Main Menu

In [4]:
# start game
def main():
    global original_screen, screen, clock, menu
    
    # initialize pygame
    pygame.init()
    pygame.font.init()

    pygame.display.set_caption('Copilot')
    original_screen = pygame.display.set_mode(SCREEN_SIZE)
    screen = original_screen.copy()
    clock = pygame.time.Clock()

    # initialize pygame_menu
    menu = pygame_menu.Menu('Copilot', SCREEN_WIDTH, SCREEN_HEIGHT, theme=pygame_menu.themes.THEME_DARK)

    # set an icon for the game
    icon = pygame.image.load('images/airplane.png')
    pygame.display.set_icon(icon)

    # create name input
    menu.add.text_input(f'Name: {BASIC_TITLE} ', default=player_name, onchange=setplayername)

    # create start button
    menu.add.button('Start', start_game, font_size=30)

    # create controls button
    menu.add.button('Controls', view_controls, font_size=30)

    # create high score button
    menu.add.button('View High Scores', view_high_scores, font_size=30)

    # create quit button
    menu.add.button('Quit', pygame_menu.events.EXIT, font_size=30)

    # main loop
    while True:
        # handle events
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.QUIT:
                return
            elif event.type == pygame_menu.events.BACK:
                return
            # elif event.type == pygame_menu.events.SELECT:
            #     menu.enable()
        # update menu
        menu.update(events)
        # draw menu
        # original_screen.fill(BACKGROUND_COLOR)
        menu.draw(original_screen)
        pygame.display.update()
        clock.tick(FPS)

In [5]:
main()

Loading scores...
Saving 11 scores...


: 

: 